# Indexing Reward Unit Test
Performing verification of the indexing rewards mechanism by comparing the event data to the results of cadCAD model.

In [1]:
import pandas as pd
import numpy as np
from decimal import Decimal
from decimal import *
import model.parts.utils as utils
from model.sys_params import *
df = pd.read_pickle(r'experiment.p')

loaded another_indexer/0xd133fd8e0607f5d82c91626140495ea0a31d0398/SIstakeDelegateds.csv.
loaded another_indexer/0xd133fd8e0607f5d82c91626140495ea0a31d0398/SIstakeLockeds.csv.
loaded another_indexer/0xd133fd8e0607f5d82c91626140495ea0a31d0398/SIstakeWithdrawns.csv.
loaded another_indexer/0xd133fd8e0607f5d82c91626140495ea0a31d0398/SIallocationCloseds.csv.
loaded another_indexer/0xd133fd8e0607f5d82c91626140495ea0a31d0398/SIallocationCollecteds.csv.
loaded another_indexer/0xd133fd8e0607f5d82c91626140495ea0a31d0398/SIstakeDepositeds.csv.
loaded another_indexer/0xd133fd8e0607f5d82c91626140495ea0a31d0398/SIrewardsAssigneds.csv.


Configure environment

In [2]:
df.reset_index(inplace = True)
pd.set_option('display.max_rows', None)

In [3]:
stake_deposited_events_dict = stake_deposited_events[0]
print("EXPECTED TRUE INDEXING REWARDS FROM STAKE DEPOSITED EVENTS:")
is_first = True
for timestep, indexing_fee_events_list in stake_deposited_events_dict.items():
    if is_first:
        is_first = False
        pass
    else:        
        if indexing_fee_events_list is None:
            indexing_fee_amt = 0
        else:
            indexing_fee_amt = sum([e['tokens'] for e in indexing_fee_events_list])        
        print(indexing_fee_amt)
        # print(f"{timestep}, {stake_deposited_event[0]['tokens']}")
    


EXPECTED TRUE INDEXING REWARDS FROM STAKE DEPOSITED EVENTS:
6685.642545872035705186
5533572818.55335728180195


In [4]:
print("MODELED RESULTS")
# print(df.iloc[timestep])
is_first = True
indexing_rewards_modeled = {}
for timestep, stake_deposited_event in stake_deposited_events_dict.items():
    # back indexing rewards out from increase in pool_delegated_stake / 0.11 * 0.89
    if is_first:
        is_first = False
        pass
    else:       
        indexing_rewards =( df.iloc[timestep-1]['pool_delegated_stake'] - df.iloc[timestep-2]['pool_delegated_stake']) / (1 - indexer_revenue_cut[0]) * (indexer_revenue_cut[0])
        indexing_rewards_modeled[timestep] = indexing_rewards
        print(f"{timestep}, {indexing_rewards=}")


MODELED RESULTS


InvalidOperation: [<class 'decimal.DivisionUndefined'>]

In [5]:
print("UNITTEST RESULTS")
is_first = True
for timestep, stake_deposited_event in stake_deposited_events_dict.items():
    # back indexing rewards out from increase in pool_delegated_stake / 0.11 * 0.89
    if is_first:
        is_first = False
        pass
    else:       
        indexing_rewards = indexing_rewards_modeled[timestep]
        print(f"Event: Timestep={timestep-1}, Indexing Reward Tokens={stake_deposited_event[0]['tokens']}")
        print(f"Model: Timestep={timestep-1}, Indexing Reward Tokens={indexing_rewards}")
        print("Tokens Equal?", stake_deposited_event[0]['tokens'] == indexing_rewards)
        print("Tokens Within 1%?", 0.99 <= abs(stake_deposited_event[0]['tokens'] / indexing_rewards) <= 1.01)
        print()


UNITTEST RESULTS


KeyError: 105

# Conclusion
The indexing reward token quantities are accurate to roughly 21 significant figures except in timestep 58.  